In [1]:
import numpy as np
import pickle
import random
import copy
import os
import shutil

import nltk
from nltk.corpus import words

In [2]:
def filter_largest_websites(website_metadata, num_websites):
    filtered_metadata = {}
    website_labels = []
    website_sizes = []
    for website, data in website_metadata.items():
        total_size = data['size'] + sum(resource['size'] for resource in data['resource_data'])
        website_labels.append(website)
        website_sizes.append(total_size)
        
    
    size_positions = np.argsort(website_sizes)
    
    count = 0
    for position in size_positions[::-1]:
        if count == num_websites:
            break
        filtered_metadata[website_labels[position]] = website_metadata[website_labels[position]]
        count+=1
        
    return filtered_metadata

In [3]:
with open('website_metadata', 'rb') as f:
    website_metadata = pickle.load(f)
    
filtered_metadata = filter_largest_websites(website_metadata, 100)
    
nltk.download('words')
word_list = [word for word in words.words() if word.isalpha()]

[nltk_data] Downloading package words to
[nltk_data]     /Users/michaelwrana/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
def generate_resource_file(file_path, size_bytes):    
    base_js_code = f"""
    document.addEventListener("DOMContentLoaded", function() {{
        let container = document.createElement("div");
        let header = document.createElement("div");
        let title = document.createElement("h3");
        let button = document.createElement("button");
        let content = document.createElement("div");

        title.innerText = "{file_path}";
        button.innerText = "Toggle Content";
        content.style.display = "none";

        button.onclick = function() {{
            content.style.display = content.style.display === "none" ? "block" : "none";
        }};

        header.style.display = "flex";
        header.style.alignItems = "center";
        header.style.gap = "10px";

        header.appendChild(title);
        header.appendChild(button);
        
        let text = "This is a new bold paragraph. "
    """
    
    while len(base_js_code.encode("utf-8")) < size_bytes - 50:
        base_js_code += f' + " {random.choice(word_list)}"'
    
    base_js_code += """;
        content.innerHTML = "<b>" + text + "</b>";
        content.style.display = "none"; // Ensure it remains minimized on page load

        container.appendChild(header);
        container.appendChild(content);
        document.body.appendChild(container);
    });
    """
    
    with open(file_path, "w") as file:
        file.write(base_js_code)
    
    print(f"JavaScript file '{file_path}' has been created successfully with size {size_bytes} bytes.")

In [5]:
def generate_html_page(file_path, size_bytes, num_resources, resource_index):
    html_template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>{file_path}</title>
        <style>
            #content {{
                display: none;
            }}
            .title-container {{
                display: flex;
                align-items: center;
                gap: 10px;
                margin-bottom: 10px;
            }}
        </style>
        <script>
            function toggleContent() {{
                var content = document.getElementById('content');
                if (content.style.display === 'none') {{
                    content.style.display = 'block';
                }} else {{
                    content.style.display = 'none';
                }}
            }}
        </script>
    </head>
    <body>
    <div class="title-container">
        <h3>{file_path}</h3>
        <button onclick='toggleContent()'>Toggle Content</button>
    </div>
    """
    
    resource_links = "".join([f'<script src="resource_{i+resource_index}.js"></script>' for i in range(num_resources)])
    
    footer = """
    </div>
    </body>
    </html>
    """
    
    current_size = len(html_template) + len(resource_links) + len(footer)
    
    words = []
    while current_size < size_bytes - len(footer):  # Ensure room for footer
        word = random.choice(word_list)
        words.append(word)
        current_size += len(word) + 1  # +1 for space
    
    content = ' '.join(words)
    full_html = f"{html_template}{resource_links}<div id='content'>{content}</div>{footer}"
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(full_html)  # Ensure full valid HTML
    
    print(f"HTML page generated: {file_path} ({os.path.getsize(file_path)} bytes)")

In [8]:
'''
overlap_all: amount of data to be shared by every website (in bytes)
overlap_group: amount of data to be shared by each website group (in bytes)
num_groups: number of website groups
num_servers: number of servers
'''
def configure_websites(website_metadata, num_servers, generate_html=True, generate_resources=False,verbose=False):
    website_to_resource = {} # mapping websites to the resources that are part of them
    server_to_website = {} # mapping servers to the websites they are hosting
    server_to_resource = {} # mapping servers to the resources they are hosting
    
    for i in range(num_servers):
        server_to_website[f"server_{i}"] = []
        server_to_resource[f"server_{i}"] = []
    
    for i in range(len(website_metadata)):
        website_to_resource[f"website_{i}"] = []
    
    server_counter = 0
    resource_counter = 0
    website_counter = 0
    
    for _, metadata in website_metadata.items():
        
        server_to_website[f'server_{server_counter}'].append(f'website_{website_counter}')
        if generate_html:
            os.makedirs(f'websites/website_{website_counter}', exist_ok=True)
            generate_html_page(
             file_path=f'websites/website_{website_counter}/index.html', 
             size_bytes=metadata['size'], 
             num_resources=len(metadata['resource_data']), 
             resource_index=resource_counter
            )
        for resource in metadata['resource_data']:
            server_to_resource[f'server_{server_counter}'].append(f'resource_{resource_counter}')
            website_to_resource[f'website_{website_counter}'].append(f'resource_{resource_counter}')
            if generate_resources:
                generate_resource_file(
                 file_path=f'resources/resource_{resource_counter}.js',
                 size_bytes=resource['size']
                )
            resource_counter += 1
        
        website_counter += 1
        server_counter = (server_counter + 1) % num_servers
    
    if verbose:
        print(website_to_resource)
        print(server_to_website)
        print(server_to_resource)
    
    return website_to_resource, server_to_website, server_to_resource
    

In [9]:
os.makedirs("websites", exist_ok=True)
os.makedirs("resources", exist_ok=True)

website_to_resource, server_to_website, server_to_resource = configure_websites(
    filtered_metadata, 
    num_servers=50, 
    generate_html=True,
    generate_resources=False,
    verbose=False)

with open("websites/config",'wb') as f:
    pickle.dump((website_to_resource,server_to_website,server_to_resource), f)

HTML page generated: websites/website_0/index.html (124271 bytes)
JavaScript file 'resources/resource_0' has been created successfully with size 30678 bytes.
JavaScript file 'resources/resource_1' has been created successfully with size 32914 bytes.
JavaScript file 'resources/resource_2' has been created successfully with size 47615 bytes.
JavaScript file 'resources/resource_3' has been created successfully with size 70367 bytes.
JavaScript file 'resources/resource_4' has been created successfully with size 118149 bytes.
JavaScript file 'resources/resource_5' has been created successfully with size 73866 bytes.
JavaScript file 'resources/resource_6' has been created successfully with size 74894 bytes.
JavaScript file 'resources/resource_7' has been created successfully with size 36078 bytes.
JavaScript file 'resources/resource_8' has been created successfully with size 73947 bytes.
JavaScript file 'resources/resource_9' has been created successfully with size 1416507 bytes.
JavaScript 

JavaScript file 'resources/resource_91' has been created successfully with size 361146 bytes.
JavaScript file 'resources/resource_92' has been created successfully with size 11387 bytes.
JavaScript file 'resources/resource_93' has been created successfully with size 94324 bytes.
JavaScript file 'resources/resource_94' has been created successfully with size 43324 bytes.
JavaScript file 'resources/resource_95' has been created successfully with size 49061 bytes.
JavaScript file 'resources/resource_96' has been created successfully with size 427868 bytes.
JavaScript file 'resources/resource_97' has been created successfully with size 45611 bytes.
JavaScript file 'resources/resource_98' has been created successfully with size 11814 bytes.
JavaScript file 'resources/resource_99' has been created successfully with size 10678 bytes.
JavaScript file 'resources/resource_100' has been created successfully with size 60079 bytes.
JavaScript file 'resources/resource_101' has been created successfu

JavaScript file 'resources/resource_179' has been created successfully with size 379641 bytes.
JavaScript file 'resources/resource_180' has been created successfully with size 553 bytes.
JavaScript file 'resources/resource_181' has been created successfully with size 0 bytes.
JavaScript file 'resources/resource_182' has been created successfully with size 223639 bytes.
JavaScript file 'resources/resource_183' has been created successfully with size 47958 bytes.
JavaScript file 'resources/resource_184' has been created successfully with size 107562 bytes.
JavaScript file 'resources/resource_185' has been created successfully with size 533988 bytes.
JavaScript file 'resources/resource_186' has been created successfully with size 284779 bytes.
JavaScript file 'resources/resource_187' has been created successfully with size 63923 bytes.
JavaScript file 'resources/resource_188' has been created successfully with size 13554 bytes.
JavaScript file 'resources/resource_189' has been created suc

JavaScript file 'resources/resource_270' has been created successfully with size 153022 bytes.
JavaScript file 'resources/resource_271' has been created successfully with size 191615 bytes.
JavaScript file 'resources/resource_272' has been created successfully with size 33 bytes.
JavaScript file 'resources/resource_273' has been created successfully with size 31 bytes.
JavaScript file 'resources/resource_274' has been created successfully with size 31 bytes.
JavaScript file 'resources/resource_275' has been created successfully with size 43 bytes.
JavaScript file 'resources/resource_276' has been created successfully with size 43 bytes.
JavaScript file 'resources/resource_277' has been created successfully with size 43 bytes.
JavaScript file 'resources/resource_278' has been created successfully with size 43 bytes.
JavaScript file 'resources/resource_279' has been created successfully with size 43 bytes.
JavaScript file 'resources/resource_280' has been created successfully with size 4

JavaScript file 'resources/resource_363' has been created successfully with size 432404 bytes.
JavaScript file 'resources/resource_364' has been created successfully with size 24017 bytes.
JavaScript file 'resources/resource_365' has been created successfully with size 120079 bytes.
JavaScript file 'resources/resource_366' has been created successfully with size 33703 bytes.
JavaScript file 'resources/resource_367' has been created successfully with size 644005 bytes.
JavaScript file 'resources/resource_368' has been created successfully with size 37041 bytes.
JavaScript file 'resources/resource_369' has been created successfully with size 36412 bytes.
JavaScript file 'resources/resource_370' has been created successfully with size 545 bytes.
JavaScript file 'resources/resource_371' has been created successfully with size 43 bytes.
JavaScript file 'resources/resource_372' has been created successfully with size 43 bytes.
JavaScript file 'resources/resource_373' has been created success

...

JavaScript file 'resources/resource_7678' has been created successfully with size 183094 bytes.
JavaScript file 'resources/resource_7679' has been created successfully with size 9227 bytes.
JavaScript file 'resources/resource_7680' has been created successfully with size 174216 bytes.
JavaScript file 'resources/resource_7681' has been created successfully with size 16420 bytes.
JavaScript file 'resources/resource_7682' has been created successfully with size 298771 bytes.
JavaScript file 'resources/resource_7683' has been created successfully with size 412911 bytes.
JavaScript file 'resources/resource_7684' has been created successfully with size 15604 bytes.
JavaScript file 'resources/resource_7685' has been created successfully with size 6767 bytes.
JavaScript file 'resources/resource_7686' has been created successfully with size 38437 bytes.
JavaScript file 'resources/resource_7687' has been created successfully with size 19646 bytes.
JavaScript file 'resources/resource_7688' has be

JavaScript file 'resources/resource_7805' has been created successfully with size 323398 bytes.
JavaScript file 'resources/resource_7806' has been created successfully with size 373141 bytes.
JavaScript file 'resources/resource_7807' has been created successfully with size 379817 bytes.
JavaScript file 'resources/resource_7808' has been created successfully with size 188934 bytes.
JavaScript file 'resources/resource_7809' has been created successfully with size 297869 bytes.
JavaScript file 'resources/resource_7810' has been created successfully with size 2356 bytes.
JavaScript file 'resources/resource_7811' has been created successfully with size 2352 bytes.
JavaScript file 'resources/resource_7812' has been created successfully with size 50 bytes.
JavaScript file 'resources/resource_7813' has been created successfully with size 345999 bytes.
JavaScript file 'resources/resource_7814' has been created successfully with size 4286 bytes.
HTML page generated: websites/website_92/index.htm

JavaScript file 'resources/resource_7897' has been created successfully with size 256786 bytes.
JavaScript file 'resources/resource_7898' has been created successfully with size 144135 bytes.
JavaScript file 'resources/resource_7899' has been created successfully with size 44 bytes.
JavaScript file 'resources/resource_7900' has been created successfully with size 43 bytes.
JavaScript file 'resources/resource_7901' has been created successfully with size 7242 bytes.
JavaScript file 'resources/resource_7902' has been created successfully with size 16958 bytes.
JavaScript file 'resources/resource_7903' has been created successfully with size 7053 bytes.
JavaScript file 'resources/resource_7904' has been created successfully with size 16958 bytes.
HTML page generated: websites/website_94/index.html (62935 bytes)
JavaScript file 'resources/resource_7905' has been created successfully with size 8531 bytes.
JavaScript file 'resources/resource_7906' has been created successfully with size 510 

JavaScript file 'resources/resource_7988' has been created successfully with size 592882 bytes.
JavaScript file 'resources/resource_7989' has been created successfully with size 31309 bytes.
JavaScript file 'resources/resource_7990' has been created successfully with size 113458 bytes.
JavaScript file 'resources/resource_7991' has been created successfully with size 448257 bytes.
JavaScript file 'resources/resource_7992' has been created successfully with size 302400 bytes.
JavaScript file 'resources/resource_7993' has been created successfully with size 35 bytes.
JavaScript file 'resources/resource_7994' has been created successfully with size 35 bytes.
JavaScript file 'resources/resource_7995' has been created successfully with size 63819 bytes.
JavaScript file 'resources/resource_7996' has been created successfully with size 117374 bytes.
JavaScript file 'resources/resource_7997' has been created successfully with size 60748 bytes.
JavaScript file 'resources/resource_7998' has been 

JavaScript file 'resources/resource_8079' has been created successfully with size 462084 bytes.
JavaScript file 'resources/resource_8080' has been created successfully with size 61080 bytes.
HTML page generated: websites/website_98/index.html (2663 bytes)
JavaScript file 'resources/resource_8081' has been created successfully with size 572749 bytes.
JavaScript file 'resources/resource_8082' has been created successfully with size 9330 bytes.
JavaScript file 'resources/resource_8083' has been created successfully with size 11174 bytes.
JavaScript file 'resources/resource_8084' has been created successfully with size 37908 bytes.
JavaScript file 'resources/resource_8085' has been created successfully with size 223773 bytes.
JavaScript file 'resources/resource_8086' has been created successfully with size 356645 bytes.
JavaScript file 'resources/resource_8087' has been created successfully with size 2549 bytes.
JavaScript file 'resources/resource_8088' has been created successfully with s